In [1]:
import pandas as pd
from openpyxl import load_workbook
import glob
from pathlib import Path
pd.set_option('display.expand_frame_repr', False)  # Evita que las filas se dividan
pd.set_option('display.max_columns', None)  # Muestra todas las columnas


### Lectura de Config

In [2]:
wb = load_workbook('config.xlsx')
ws = wb.active
valor = ws.cell(row=3, column=2).value

carpeta = Path(valor) 
archivo = list(carpeta.rglob("CRM*.xlsx")) + list(carpeta.glob("CRM*.xls")) # se filtra por CRM xlsx u xls
archivos_str = [str(a) for a in archivo] # retorna un array ['xxx','xxxx',...]de las ruta de los archivos

### Generacion de Hoja CRM

In [ ]:
def leer_excel_smart(path, hoja_principal="Base CRM", hoja_backup="Hoja1"):
    """
    Lee un Excel detectando:
    - qué hoja usar (principal o backup)
    - primera fila válida como encabezado
    - limpia filas vacías
    - estandariza nombres de columnas
    """

    column_map = {
        "No.": "numero",
        "No": "numero",
        "N°": "numero",
        "nro": "numero",

        "Fecha": "fecha",
        "Asesor": "asesor",
        "Contacto": "contacto",
        "Cargo": "cargo",

        "teléfono": "telefono",
        "telefono": "telefono",
        "Teléfono": "telefono",

        "Mail": "mail",
        "Correo": "mail",

        "Razón Social": "razon_social",
        "Razon Social": "razon_social",

        "RUC": "ruc",
        "Web": "web",
        "Departamento": "departamento",
        "Distrito": "distrito",
        "Rubro": "rubro",
        "Tipo de producto": "tipo_producto",
        "Tipo de Producto": "tipo_producto",

        "Origen del lead": "origen_lead",
        "Descripción": "descripcion",
        "Estado": "estado",
        "Monto de la oportunidad": "monto_oportunidad",
        "No. Cotización": "numero_cotizacion",

        "Valor Cotización": "valor_cotizacion",
        "Valor Cotización + IGV": "valor_cotizacion",

        "Fecha seguimiento": "fecha_seguimiento",
        "Semana": "semana",
    }

    try:
        with pd.ExcelFile(path) as xls:
            if hoja_principal in xls.sheet_names:
                hoja = hoja_principal
            elif hoja_backup in xls.sheet_names:
                hoja = hoja_backup
            else:
                print(f"❌ Ninguna hoja válida encontrada en: {path}")
                return None

            df_raw = pd.read_excel(xls, sheet_name=hoja, header=None)
    except Exception as e:
        print(f"❌ Error al abrir archivo {path}: {e}")
        return None

    header_row = next((i for i in range(len(df_raw)) if df_raw.iloc[i].notna().any()), None) # busca la primera fila que no esta vacia

    if header_row is None:
        print(f"❌ No se encontró encabezado en: {path}")
        return None

    df = pd.read_excel(path, sheet_name=hoja, header=header_row)

    df = df.dropna(how='all').reset_index(drop=True) # quita las fila completamente vacias

    # Estandarizar columnas
    df.columns = df.columns.astype(str).str.strip()  # quita espacios
    
    df = df.rename(columns=lambda c: column_map.get(c, c.lower().replace(" ", "_")))
    
    return df



In [12]:
archivo_lectura = []
# archivo_cierre = []
for archivo  in archivos_str:
    df_1 = leer_excel_smart(archivo, hoja_principal="Base CRM", hoja_backup="CRM")
    if df_1 is None:
        print(f"Saltando archivo no válido: {archivo}")
        continue
    archivo_lectura.append(df_1)
    
    

c:\Users\Gonzalo PL\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Gonzalo PL\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Gonzalo PL\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Gonzalo PL\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Gonzalo PL\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Go

In [13]:
df_fina = pd.concat(archivo_lectura, ignore_index=True) # creamos un dataframe 

In [8]:
with pd.ExcelWriter("salida.xlsx", engine="openpyxl") as writer:
    df_fina.to_excel(writer, index=False)
    wb = writer.book
    ws = writer.sheets['Sheet1']
    
    for col in ['B','U']:
        for cell in ws[col]:
            if cell.value is not None:
                cell.number_format = "DD/MM/YYYY"
            
    for col in ['D','E','F']:
        for cell in ws[col]:
            if cell.value is not None:
                cell.number_format = "@"
              
    

### Procesamiento datos de cierre

In [19]:
# desarrollo de cierre
datosCierre = []
for archivo in archivos_str:
    # desarrollo de cierre
    claves = ['No.','Fecha','Asesor','Razon Social','RUC']
    pattern = "|".join(claves)
    dfCierre = pd.read_excel(archivo, sheet_name='CIERRE', header=None)

    # Buscar fila donde están los encabezados
    filaCierre = dfCierre.index[
        dfCierre.apply(lambda row: row.astype(str).str.contains(pattern, case=False).any(), axis=1)
    ][0]

    print("Fila encontrada:", filaCierre)

    # Cargar excel usando esa fila como encabezado
    df_final = pd.read_excel(
        archivo,
        sheet_name='CIERRE',
        header=filaCierre
    )

    datosCierre.append(df_final)
    
dfCierres = pd.concat(datosCierre, ignore_index=True)
print(dfCierres)

Fila encontrada: 2
Fila encontrada: 2
Fila encontrada: 2
   No.      Fecha  Asesor               Razon Social           RUC  Código                                    Producto       Modelo             KG Marca  Costo     Sub Total          Igv      Total  Costos      fecha
0    1 2025-11-13     NaN    Quiñones Alvarez Angelo           NaN  I22471                            Centro de Lavado  Giant C Max     10 a 13 KG    LG    NaN   3474.580000   625.424000   4100.004     NaN        NaT
1    2 2025-11-15     NaN        Seventh Heaven SCRL  2.060165e+10  I23000  Rodillo de planchado industrial automatico       G14.25  1450 x 270 mm   GMP    NaN  14406.779661  2593.220339  17000.000     NaN        NaT
2    3 2025-11-19     NaN  Corporacion Munakuyki SAC  2.061362e+10  I25103         Lavadora centrifuga semi industrial    Titan Max          17 KG    LG    NaN   3950.000000   711.000000   4661.000     NaN        NaT
3    1 2025-11-13     NaN    Quiñones Alvarez Angelo           NaN  I22471 

In [16]:
listProductos = pd.read_excel('concatenacion_base_precios_1.xlsx', sheet_name='Sheet1')

In [21]:
dfMerge = pd.merge(dfCierres, listProductos, left_on="Código", right_on="CÓDIGO", how='left')
dfMerge = dfMerge.drop(columns={'CÓDIGO','NOMBRE COMERCIAL'})
dfMerge

,No.,Fecha,Asesor,Razon Social,RUC,Código,Producto,Modelo,KG,Marca,Costo,Sub Total,Igv,Total,Costos,fecha,COSTO COMPASS
0,1,2025-11-13,NaN,Quiñones Alvarez Angelo,NaN,I22471,Centro de Lavado,Giant C Max,10 a 13 KG,LG,NaN,3474.580000,625.424000,4100.004,NaN,NaT,30750.000
1,2,2025-11-15,NaN,Seventh Heaven SCRL,2.060165e+10,I23000,Rodillo de planchado industrial automatico,G14.25,1450 x 270 mm,GMP,NaN,14406.779661,2593.220339,17000.000,NaN,NaT,1635.226
2,3,2025-11-19,NaN,Corporacion Munakuyki SAC,2.061362e+10,I25103,Lavadora centrifuga semi industrial,Titan Max,17 KG,LG,NaN,3950.000000,711.000000,4661.000,NaN,NaT,3193.500
3,1,2025-11-13,NaN,Quiñones Alvarez Angelo,NaN,I22471,Centro de Lavado,Giant C Max,10 a 13 KG,LG,NaN,3474.580000,625.424000,4100.004,NaN,NaT,30750.000
4,2,2025-11-15,NaN,Seventh Heaven SCRL,2.060165e+10,I23000,Rodillo de planchado industrial automatico,G14.25,1450 x 270 mm,GMP,NaN,14406.779661,2593.220339,17000.000,NaN,NaT,1635.226
5,3,2025-11-19,NaN,Corporacion Munakuyki SAC,2.061362e+10,I25103,Lavadora centrifuga semi industrial,Titan Max,17 KG,LG,NaN,3950.000000,711.000000,4661.000,NaN,NaT,3193.500
6,1,NaT,NaN,Quiñones Alvarez Angelo,NaN,I22471,Centro de Lavado,Giant C Max,10 a 13 KG,LG,NaN,3474.580000,625.424000,4100.004,NaN,2025-11-13,30750.000
7,2,NaT,NaN,Seventh Heaven SCRL,2.060165e+10,I23000,Rodillo de planchado industrial automatico,G14.25,1450 x 270 mm,GMP,NaN,14406.779661,2593.220339,17000.000,NaN,2025-11-15,1635.226
8,3,NaT,NaN,Corporacion Munakuyki SAC,2.061362e+10,I25103,Lavadora centrifuga semi industrial,Titan Max,17 KG,LG,NaN,3950.000000,711.000000,4661.000,NaN,2025-11-19,3193.500
